In [13]:
# pip install --upgrade git+https://github.com/huggingface/transformers.git
from transformers import pipeline
import comments
import pandas as pd

In [2]:
classifier = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    return_all_scores=True,
)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
/Users/vrishank/Comp Sci Projects/Portfolio Projects/CommentSense/backend/.venv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [31]:
video_id = "JTOJsU3FSD8"
vid_comments = comments.get_video_comments(video_id)
vid_comments[0]

'Or just build a todo list.'

In [38]:
predictions = classifier(
    vid_comments
)

In [43]:
predictions[0]

[{'label': 'sadness', 'score': 0.01638314686715603},
 {'label': 'joy', 'score': 0.3776950538158417},
 {'label': 'love', 'score': 0.005642768926918507},
 {'label': 'anger', 'score': 0.5723163485527039},
 {'label': 'fear', 'score': 0.025497302412986755},
 {'label': 'surprise', 'score': 0.002465476980432868}]

In [60]:
df_data = [
    {**{'comment': comment}, **{item['label']: item['score'] for item in prediction}}
    for comment, prediction in zip(vid_comments, predictions)
]
df = pd.DataFrame(df_data)

df.head()

,comment,sadness,joy,love,anger,fear,surprise
0,Or just build a todo list.,0.016383,0.377695,0.005643,0.572316,0.025497,0.002465
1,this = 🔥,0.096259,0.404669,0.013005,0.209291,0.259173,0.017603
2,At 0:35 what is the name of the software on sc...,0.020735,0.082521,0.003139,0.352079,0.536158,0.005367
3,https://www.youtube.com/watch?v=_VRT2FFXntc&t,0.031786,0.559257,0.012082,0.117376,0.273846,0.005653
4,Or just build a weather app,0.014391,0.895009,0.002933,0.052879,0.032984,0.001804


In [61]:
sadAvg = df['sadness'].mean()
angryAvg = df['anger'].mean()
happyAvg = df['joy'].mean()
fearAvg = df['fear'].mean()
surpriseAvg = df['surprise'].mean()

totals = {
    'sadness': sadAvg,
    'anger': angryAvg,
    'joy': happyAvg,
    'fear': fearAvg,
    'surprise': surpriseAvg
}

In [64]:
result = []

for comment, pred in zip(vid_comments, predictions):
    sentiment_scores = {label['label']: label['score'] for label in pred}
    comment_dict = {'comment': comment, 'sentiment_scores': sentiment_scores}
    result.append(comment_dict)

result

returnVal = {}
returnVal['averages'] = totals
returnVal['result'] = result
returnVal


{'averages': {'sadness': 0.059394523613015365,
  'anger': 0.2394989188054724,
  'joy': 0.5588304264485541,
  'fear': 0.10230343057364155,
  'surprise': 0.021828071972075546},
 'result': [{'comment': 'Or just build a todo list.',
   'sentiment_scores': {'sadness': 0.01638314686715603,
    'joy': 0.3776950538158417,
    'love': 0.005642768926918507,
    'anger': 0.5723163485527039,
    'fear': 0.025497302412986755,
    'surprise': 0.002465476980432868}},
  {'comment': 'this = 🔥',
   'sentiment_scores': {'sadness': 0.09625854343175888,
    'joy': 0.40466931462287903,
    'love': 0.013004631735384464,
    'anger': 0.2092914879322052,
    'fear': 0.25917306542396545,
    'surprise': 0.017603036016225815}},
  {'comment': 'At 0:35 what is the name of the software on screen please??',
   'sentiment_scores': {'sadness': 0.0207352451980114,
    'joy': 0.0825209990143776,
    'love': 0.003139213426038623,
    'anger': 0.35207921266555786,
    'fear': 0.5361582636833191,
    'surprise': 0.00536704